In [1]:
import pandas as pd
from glob import glob
import numpy as np
import cv2
from scipy.spatial.transform import Rotation as R

In [23]:
'''
Frames -
    CameraLink - Final camera link frame on the robot's left camera transformed to orb's requirements
    Map - First frame for orb
    World - First frame for kitti
    CameraLinkKitti - Camera link frame on the robot's left camera(cam0)
'''

#def convert_to_world()
def convert_posevec2transformation(pose):
    t,quat=pose[:3],pose[3:]
    rot_mat = R.from_quat(quat).as_matrix()
    T=np.hstack((rot_mat,t.reshape(3,1)))
    T=np.vstack((T,np.zeros((1,4))))
    T[-1,-1]=1
    return T
def convert_transformation2posevec(T):
    T/=T[-1,-1]
    rot_mat = T[:3,:3]
    t = T[:3,3]
    quat = R.from_matrix(rot_mat).as_quat()
    pose = np.concatenate((t,quat))
    return pose
    
    
def get_static_tf(orb_pose_df,n=0):
    statictf_world2camera_link_kitti = orb_pose_df[orb_pose_df['child_frame_id']=='"camera_link_kitti"'].iloc[n][['x','y','z','qx','qy','qz','qw']].values
    statictf_camera_link_kitti2map = orb_pose_df[orb_pose_df['child_frame_id']=='"map"'].iloc[n][['x','y','z','qx','qy','qz','qw']].values
    T_camkitti2world=convert_posevec2transformation(statictf_world2camera_link_kitti).copy()
    T_map2camkitti=convert_posevec2transformation(statictf_camera_link_kitti2map).copy()
    return T_camkitti2world,T_map2camkitti

def get_orb_poses_world(orb_pose_df):
    orb_poses_df_map = orb_pose_df[orb_pose_df['child_frame_id']=='"camera_link"']
    orb_poses_in_map = orb_poses_df_map[['x','y','z','qx','qy','qz','qw']].values
    T_camkitti2world,T_map2camera_link_kitti = get_static_tf(orb_pose_df)
    
    orb_poses_in_world=np.empty(orb_poses_in_map.shape)
    
    orb_poses_df_world = pd.DataFrame().reindex_like(orb_poses_df_map)
    
    for idx,orb_pose in enumerate(orb_poses_in_map):
        index = orb_poses_df_map.iloc[[idx]].index
        T_camlink2map = convert_posevec2transformation(orb_pose).copy()
        T_camlink2world = T_camkitti2world @ T_map2camera_link_kitti @ T_camlink2map
        orb_poses_in_world[idx] = convert_transformation2posevec(T_camlink2world)
        orb_poses_df_world.loc[index,['x','y','z','qx','qy','qz','qw']] = orb_poses_in_world[idx]
    
    orb_poses_df_world[['child_frame_id','rosbagTimestamp','secs','nsecs']] = orb_poses_df_map[['child_frame_id','rosbagTimestamp','secs','nsecs']].copy()
    orb_poses_df_world['frame_id']= '"world"'
    return orb_poses_in_world,orb_poses_df_world
    

    


In [21]:
index=orb_pose_df_map.iloc[[idx]].index
orb_poses_df_world.loc[index,['x']]
# orb_poses_in_world.shape

NameError: name 'orb_pose_df_map' is not defined

In [14]:
orb_pose_df = get_orb_pose_df()
orb_poses_in_world,orb_poses_df_world = get_orb_poses_world(orb_pose_df)
orb_poses_df_world.to_csv('../data/orb_poses_world_full.csv')


NameError: name 'orb_pose_df_map' is not defined

In [35]:
def create_merged_df():
    # Record Orb poses bag first by recording tf on the resulting poses and aling columns correctly and remove empty columns
    orb_poses_df = pd.read_csv('../data/orb_poses_full_removed_emptycolumns.csv')
    #orb_pose_df = orb_pose_df.drop(columns=['rosbagTimestamp','Unnamed: 1', 'Unnamed: 2', 'seq','Unnamed: 8', 'transform','Unnamed: 13'])
    orb_pose_df_map = orb_poses_df[orb_poses_df['child_frame_id']=='"camera_link"']
    _,orb_poses_df_world = get_orb_poses_world(orb_poses_df)
    kitti_images_df_short = pd.read_csv('../data/kitti_images_nodata.csv')
    
    merged_df = pd.merge(orb_poses_df,kitti_images_df_short,on=['nsecs','secs'])
    merged_df_world = pd.merge(orb_poses_df_world,kitti_images_df_short,on=['nsecs','secs'])
    
    merged_df_world.to_csv('../data/kitti_data_orb_poses_world_full.csv')
    merged_df.to_csv('../data/kitti_data_orb_poses_full.csv')
    
    merged_df = merged_df[merged_df.frame_id_x=='"world"']
    merged_df_world = merged_df_world[merged_df_world.frame_id_x=='"world"']
    return merged_df,merged_df_world
merged_df,merged_df_world=create_merged_df()
merged_df_world

,rosbagTimestamp,secs,nsecs,frame_id_x,child_frame_id,x,y,z,qx,qy,qz,qw,Unnamed: 0,frame_id_y,files
0,1.606005e+18,1603863769,607414007,"""world""","""camera_link""",-0.002140,0.003377,0.256551,0.002018,0.707415,-0.004362,-0.706782,5,"""camera_color_left""",../data/images/frame0005.jpg
1,1.606005e+18,1603863769,711162090,"""world""","""camera_link""",-0.003418,0.003741,0.309299,0.003391,0.707841,-0.004533,-0.706349,6,"""camera_color_left""",../data/images/frame0006.jpg
2,1.606005e+18,1603863769,814770936,"""world""","""camera_link""",-0.003461,0.004598,0.363476,0.004362,0.709002,-0.004702,-0.705178,7,"""camera_color_left""",../data/images/frame0007.jpg
3,1.606005e+18,1603863769,918503999,"""world""","""camera_link""",-0.003862,0.005624,0.418211,0.006647,0.709738,-0.004146,-0.704422,8,"""camera_color_left""",../data/images/frame0008.jpg
4,1.606005e+18,1603863770,22125005,"""world""","""camera_link""",-0.004750,0.006628,0.472209,0.007176,0.709423,-0.004380,-0.704733,9,"""camera_color_left""",../data/images/frame0009.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
954,1.606005e+18,1603863868,510201930,"""world""","""camera_link""",0.013510,13.915910,23.713111,0.711659,-0.005023,-0.702135,-0.022856,959,"""camera_color_left""",../data/images/frame0959.jpg
955,1.606005e+18,1603863868,614001989,"""world""","""camera_link""",0.013257,13.915311,23.657022,0.711037,-0.009755,-0.702882,-0.016987,960,"""camera_color_left""",../data/images/frame0960.jpg
956,1.606005e+18,1603863868,717802047,"""world""","""camera_link""",0.014112,13.914068,23.600803,0.710894,-0.013585,-0.703077,-0.011314,961,"""camera_color_left""",../data/images/frame0961.jpg
957,1.606005e+18,1603863868,821702003,"""world""","""camera_link""",0.013229,13.912120,23.544786,0.711605,-0.017179,-0.702336,-0.006949,962,"""camera_color_left""",../data/images/frame0962.jpg


In [30]:
kitti_images_df_short = pd.read_csv('../data/kitti_images_nodata.csv')
kitti_images_df_short

,Unnamed: 0,secs,nsecs,frame_id,files
0,0,1603863769,89112043,"""camera_color_left""",../data/images/frame0000.jpg
1,1,1603863769,192698955,"""camera_color_left""",../data/images/frame0001.jpg
2,2,1603863769,296443939,"""camera_color_left""",../data/images/frame0002.jpg
3,3,1603863769,400181055,"""camera_color_left""",../data/images/frame0003.jpg
4,4,1603863769,503798007,"""camera_color_left""",../data/images/frame0004.jpg
...,...,...,...,...,...
959,959,1603863868,510201930,"""camera_color_left""",../data/images/frame0959.jpg
960,960,1603863868,614001989,"""camera_color_left""",../data/images/frame0960.jpg
961,961,1603863868,717802047,"""camera_color_left""",../data/images/frame0961.jpg
962,962,1603863868,821702003,"""camera_color_left""",../data/images/frame0962.jpg


In [16]:
merged_df

,rosbagTimestamp,secs,nsecs,frame_id_x,child_frame_id,x,y,z,qx,qy,qz,qw,Unnamed: 0,frame_id_y,files
6,1.606005e+18,1603863769,607414007,"""map""","""camera_link""",0.256551,0.003377,0.002140,0.001657,-0.000448,0.004511,0.999988,5,"""camera_color_left""",../data/images/frame0005.jpg
8,1.606005e+18,1603863769,711162090,"""map""","""camera_link""",0.309299,0.003741,0.003418,0.000807,-0.001055,0.005604,0.999983,6,"""camera_color_left""",../data/images/frame0006.jpg
10,1.606005e+18,1603863769,814770936,"""map""","""camera_link""",0.363476,0.004598,0.003461,0.000240,-0.002704,0.006409,0.999976,7,"""camera_color_left""",../data/images/frame0007.jpg
12,1.606005e+18,1603863769,918503999,"""map""","""camera_link""",0.418211,0.005624,0.003862,-0.001769,-0.003758,0.007632,0.999962,8,"""camera_color_left""",../data/images/frame0008.jpg
14,1.606005e+18,1603863770,22125005,"""map""","""camera_link""",0.472209,0.006628,0.004750,-0.001977,-0.003316,0.008171,0.999959,9,"""camera_color_left""",../data/images/frame0009.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1914,1.606005e+18,1603863868,510201930,"""map""","""camera_link""",23.713111,13.915910,-0.013510,-0.006734,0.019713,0.999703,0.012610,959,"""camera_color_left""",../data/images/frame0959.jpg
1916,1.606005e+18,1603863868,614001989,"""map""","""camera_link""",23.657022,13.915311,-0.013257,-0.005767,0.018909,0.999792,0.005114,960,"""camera_color_left""",../data/images/frame0960.jpg
1918,1.606005e+18,1603863868,717802047,"""map""","""camera_link""",23.600803,13.914068,-0.014112,-0.005528,0.017607,0.999828,-0.001606,961,"""camera_color_left""",../data/images/frame0961.jpg
1920,1.606005e+18,1603863868,821702003,"""map""","""camera_link""",23.544786,13.912120,-0.013229,-0.006554,0.017061,0.999807,-0.007234,962,"""camera_color_left""",../data/images/frame0962.jpg


In [11]:
def get_orb_pose_df():
    orb_pose_df = pd.read_csv('../data/orb_poses_full_removed_emptycolumns.csv')
    return orb_pose_df
def get_kitti_gt_poses_df():
    kitti_gt_poses_df=pd.read_csv('../data/gt_poses_00.txt',sep=" ",header=None,nrows=300)
    kitti_gt_poses_df.columns=np.arange(1,13)
    return kitti_gt_poses_df
df=get_kitti_gt_poses_df()
df

,1,2,3,4,5,6,7,8,9,10,11,12
0,1.000000,9.043680e-12,2.326809e-11,5.551115e-17,9.043683e-12,1.000000,2.392370e-10,3.330669e-16,2.326810e-11,2.392370e-10,1.000000,-4.440892e-16
1,0.999998,5.272628e-04,-2.066935e-03,-4.690294e-02,-5.296506e-04,0.999999,-1.154865e-03,-2.839928e-02,2.066324e-03,1.155958e-03,0.999997,8.586941e-01
2,0.999991,1.048972e-03,-4.131348e-03,-9.374345e-02,-1.058514e-03,0.999997,-2.308104e-03,-5.676064e-02,4.128913e-03,2.312456e-03,0.999989,1.716275e+00
3,0.999980,1.566466e-03,-6.198571e-03,-1.406429e-01,-1.587952e-03,0.999993,-3.462706e-03,-8.515762e-02,6.193102e-03,3.472479e-03,0.999975,2.574964e+00
4,0.999964,2.078471e-03,-8.263498e-03,-1.874858e-01,-2.116664e-03,0.999987,-4.615826e-03,-1.135202e-01,8.253797e-03,4.633149e-03,0.999955,3.432648e+00
...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.990926,4.849401e-02,1.253563e-01,7.105219e+01,-4.546236e-02,0.998603,-2.693473e-02,-7.868414e+00,-1.264873e-01,2.099133e-02,0.991746,1.538549e+02
296,0.992049,4.708364e-02,1.167168e-01,7.114828e+01,-4.394991e-02,0.998605,-2.928025e-02,-7.891249e+00,-1.179326e-01,2.391773e-02,0.992733,1.546649e+02
297,0.992957,4.708196e-02,1.087220e-01,7.123668e+01,-4.431524e-02,0.998633,-2.772649e-02,-7.916918e+00,-1.098787e-01,2.271316e-02,0.993685,1.554826e+02
298,0.993593,5.028742e-02,1.012092e-01,7.130611e+01,-4.847236e-02,0.998618,-2.031541e-02,-7.939937e+00,-1.020909e-01,1.527941e-02,0.994658,1.563078e+02


In [3]:
def get_static_tf_transform(orb_first_frame_no=0):
    kitti_gt_poses_file = open('../data/gt_poses_00.txt')
    kitti_gt_poses = kitti_gt_poses_file.readlines()
    kitti_gt_poses = kitti_gt_poses[orb_first_frame_no].split(' ')
    map_world_frame = np.array(kitti_gt_poses).astype(np.float32).reshape(3,4)
    rot,trans=map_world_frame[:3,:3],map_world_frame[:3,3]
    quat = R.from_matrix(rot).as_quat()
    static_tf = np.concatenate((trans,quat)) #x,y,z,qx,qy,qz,qw
    print(static_tf)
    
    
get_static_tf_transform()
    
    

[ 5.55111512e-17  3.33066907e-16 -4.44089210e-16  0.00000000e+00
 -2.16840441e-18  8.67361764e-19  1.00000000e+00]


In [21]:
def process_original_kitti_df():
    kitti_images_df = pd.read_csv('../data/kitti_images.csv')
    images_path = '../data/images/*.jpg'
    images_files = glob(images_path)
    images_files.sort()
    kitti_images_df_short = kitti_images_df.drop(columns=['rosbagTimestamp','data','encoding','height','width','is_bigendian','step','header','seq','stamp'])
    kitti_images_df_short['files']=images_files
    kitti_images_df_short.to_csv('../data/kitti_images_nodata.csv')

In [22]:
merged_df = pd.read_csv('../data/kitti_data_orb_poses.csv')

In [24]:
## Read data of the sequence from a loop
for index, entry in merged_df.iterrows():
    imagepath = entry['files']
    image = cv2.imread(imagepath)
    orb_pose = entry[['x','y','z','qx','qy','qz','qw']].values
    t,q = orb_pose[:3],orb_pose[3:]
    break

In [37]:
def get_kitti_gt_poses_df():
    kitti_gt_poses_df=pd.read_csv('../data/gt_poses_00.txt',sep=" ",header=None,nrows=300)
    kitti_gt_poses_df.columns=np.arange(1,13)
    return kitti_gt_poses_df
df=get_kitti_gt_poses_df()
df.iloc[:10]

,1,2,3,4,5,6,7,8,9,10,11,12
0,1.000000,9.043680e-12,2.326809e-11,5.551115e-17,9.043683e-12,1.000000,2.392370e-10,3.330669e-16,2.326810e-11,2.392370e-10,1.000000,-4.440892e-16
1,0.999998,5.272628e-04,-2.066935e-03,-4.690294e-02,-5.296506e-04,0.999999,-1.154865e-03,-2.839928e-02,2.066324e-03,1.155958e-03,0.999997,8.586941e-01
2,0.999991,1.048972e-03,-4.131348e-03,-9.374345e-02,-1.058514e-03,0.999997,-2.308104e-03,-5.676064e-02,4.128913e-03,2.312456e-03,0.999989,1.716275e+00
3,0.999980,1.566466e-03,-6.198571e-03,-1.406429e-01,-1.587952e-03,0.999993,-3.462706e-03,-8.515762e-02,6.193102e-03,3.472479e-03,0.999975,2.574964e+00
4,0.999964,2.078471e-03,-8.263498e-03,-1.874858e-01,-2.116664e-03,0.999987,-4.615826e-03,-1.135202e-01,8.253797e-03,4.633149e-03,0.999955,3.432648e+00
5,0.999943,2.586172e-03,-1.033094e-02,-2.343818e-01,-2.645881e-03,0.999980,-5.770163e-03,-1.419150e-01,1.031581e-02,5.797170e-03,0.999930,4.291335e+00
6,0.999918,3.088363e-03,-1.239599e-02,-2.812195e-01,-3.174350e-03,0.999971,-6.922975e-03,-1.702743e-01,1.237425e-02,6.961759e-03,0.999899,5.148987e+00
7,0.999889,3.586305e-03,-1.446384e-02,-3.281178e-01,-3.703403e-03,0.999960,-8.077186e-03,-1.986703e-01,1.443430e-02,8.129853e-03,0.999863,6.007777e+00
8,0.999855,4.078705e-03,-1.652913e-02,-3.749547e-01,-4.231669e-03,0.999948,-9.229794e-03,-2.270290e-01,1.649063e-02,9.298401e-03,0.999821,6.865477e+00
9,0.999817,4.566671e-03,-1.859652e-02,-4.218367e-01,-4.760342e-03,0.999935,-1.038342e-02,-2.554151e-01,1.854788e-02,1.047004e-02,0.999773,7.724036e+00


In [ ]:
help(pd)